# Nom du notebook

**Définition** : Ce que je veux faire

**Formule** : 

## On commence par le chargement des données
on décommente ce dont on a besoin

In [ ]:
import pandas as pd
import numpy as np

import time
import json
import math
from tqdm.notebook import tqdm
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
from src.elasticsearch_connector import ElasticsearchConnector
#from typing import List, Dict


pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

## Si on veut lire en requêtant Elastic Search

In [ ]:
es_connector = ElasticsearchConnector(env='monolog')

QUERY = {
    "query": {
        "bool": { 
          "must": [
        
            {
              "range": {
               "logfile": {
                  "gte": "2023-01-01",
                  "lt": "2023-01-04"
               }
              }
            }

          ]
        }
      }
}

In [ ]:
logs_january_cc = es_connector.execute_query(QUERY, "logs-new")

In [ ]:
logs_january_cc[logs_january_cc["cc"].notna()]

In [ ]:
logs_january_cc.url.value_counts()


In [ ]:
logs_january_cc['url'] = logs_january_cc['url'].str.split('#').str[0].str.split('?').str[0]

In [ ]:
logs_january_cc_visits = logs_january_cc[logs_january_cc["type"]=="visit_content"]

In [ ]:
counts = logs_january_cc_visits.groupby(by="url").agg({"idVisit": "nunique"})\
    .sort_values("idVisit", ascending=False)
counts

In [ ]:
counts.to_csv("visites-cc.csv", sep=";")
